In [3]:
import tensorflow as tf


def make_tensor():
    return tf.constant([[1, 2, 3], [4, 5, 6], [7, 8, 9]])


## GPU Default

On my systems, GPU is configured, so tensors are created on **GPU by default**.


In [4]:
t = make_tensor()
t.device

Metal device set to: Apple M1 Max


'/job:localhost/replica:0/task:0/device:GPU:0'

## Manual Override to CPU


In [12]:
with tf.device('/CPU:0'):
    t = make_tensor()
t.device  # Note that once you exit the context, the tensor remains on the CPU

'/job:localhost/replica:0/task:0/device:CPU:0'

## Operations Stay on CPU


In [6]:
with tf.device('/CPU:0'):
    t = make_tensor()
t = t**2
t.device

'/job:localhost/replica:0/task:0/device:CPU:0'

## Operations Themselves Don't Move to GPU (even in GPU context)


In [10]:
with tf.device('/CPU:0'):
    t = make_tensor()
with tf.device('/GPU:0'):
    print(t.device)
    t = t**2  # The **2 operation makes a new tensor but it is still CPU despite the context
    print(t.device)
print(t.device)

/job:localhost/replica:0/task:0/device:CPU:0
/job:localhost/replica:0/task:0/device:CPU:0
/job:localhost/replica:0/task:0/device:CPU:0


## Moving to GPU


In [11]:
with tf.device('/CPU:0'):
    t = make_tensor()
with tf.device('/GPU:0'):
    t = tf.identity(
        t
    )  # This makes a new tensor without doing operations, so it successfully moves to GPU
    print(t.device)

/job:localhost/replica:0/task:0/device:GPU:0


## Mixed Device Operations


In [15]:
with tf.device('/CPU:0'):
    t = make_tensor()
with tf.device('/GPU:0'):
    u = make_tensor()
v = t + u
print(t.device)  # Original stays on the CPU
print(u.device)  # Original stays on the GPU
print(
    v.device
)  # Result is in GPU because that's preferred (presumably t had to be copied there)


/job:localhost/replica:0/task:0/device:CPU:0
/job:localhost/replica:0/task:0/device:GPU:0
/job:localhost/replica:0/task:0/device:GPU:0


## Python Code Against GPU Tensor


In [16]:
with tf.device('/GPU:0'):
    t = make_tensor()
t = t + 5
t.device

'/job:localhost/replica:0/task:0/device:GPU:0'

## Keras Conversion to GPU


In [26]:
with tf.device('/CPU:0'):
    t = make_tensor()
    input_layer = tf.keras.Input(shape=t.shape)
    output_layer = tf.keras.layers.Dense(5)(input_layer)
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    print(model(t).device)
print(model(t).device
     )  # Only the context of the model call matters (and defaults to GPU)


/job:localhost/replica:0/task:0/device:CPU:0
/job:localhost/replica:0/task:0/device:GPU:0
